In [1]:
from pyspark.sql import SparkSession # Start Spark Session
from pyspark.sql.types import * # Type changing

In [2]:
spark = SparkSession\
.builder\
.master('local[*]')\
.appName('ML')\
.getOrCreate()

In [3]:
spark

In [4]:
path = 'Data/CogsleyServices-SalaData-US.csv'

In [5]:
data = spark.read.load(path,
                      format = 'csv',
                      header = 'true',
                      inferSchema = 'true').cache()

In [6]:
data  = data.dropna()
data.take(2)

[Row(RowID=1914, OrderID=13729, OrderDate=datetime.datetime(2009, 1, 1, 0, 0), OrderMonthYear=datetime.datetime(2009, 1, 1, 0, 0), Quantity=9, Quote=1800, DiscountPct=0.08, Rate=200, SaleAmount=1640.96, CustomerName=u'Matt Bertelsons', CompanyName=u'The Priceline Group Inc.', Sector=u'Miscellaneous', Industry=u'Business Services', City=u'Bowie', ZipCode=20715, State=u'Maryland', Region=u'East', ProjectCompleteDate=datetime.datetime(2009, 1, 3, 0, 0), DaystoComplete=2, ProductKey=u'Development - Big Data', ProductCategory=u'Development', ProductSubCategory=u'Python', Consultant=u'Noah Smith', Manager=u'Allen Young', HourlyWage=59, RowCount=1, WageMargin\=u'0.71\\'),
 Row(RowID=4031, OrderID=28774, OrderDate=datetime.datetime(2009, 1, 1, 0, 0), OrderMonthYear=datetime.datetime(2009, 1, 1, 0, 0), Quantity=32, Quote=6400, DiscountPct=0.1, Rate=200, SaleAmount=5707.67, CustomerName=u'Jessica Thornton', CompanyName=u'Garmin Ltd.', Sector=u'Capital Goods', Industry=u'Industrial Machinery/Comp

In [7]:
data.printSchema()

root
 |-- RowID: integer (nullable = true)
 |-- OrderID: integer (nullable = true)
 |-- OrderDate: timestamp (nullable = true)
 |-- OrderMonthYear: timestamp (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Quote: integer (nullable = true)
 |-- DiscountPct: double (nullable = true)
 |-- Rate: integer (nullable = true)
 |-- SaleAmount: double (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZipCode: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProjectCompleteDate: timestamp (nullable = true)
 |-- DaystoComplete: integer (nullable = true)
 |-- ProductKey: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- ProductSubCategory: string (nullable = true)
 |-- Consultant: string (nullable = true)
 |-- Manager: string (nul

In [8]:
summary = data\
.select(data["OrderMonthYear"], data["SaleAmount"])\
.groupby("OrderMonthYear")\
.sum()\
.orderBy("OrderMonthYear")

In [9]:
summary.show()

+-------------------+-----------------+
|     OrderMonthYear|  sum(SaleAmount)|
+-------------------+-----------------+
|2009-01-01 00:00:00|734559.3599999996|
|2009-02-01 00:00:00|539887.7999999998|
|2009-03-01 00:00:00|559449.9600000002|
|2009-04-01 00:00:00|        614983.31|
|2009-05-01 00:00:00|637481.3899999998|
|2009-06-01 00:00:00|555516.2199999999|
|2009-07-01 00:00:00|        670807.13|
|2009-08-01 00:00:00|660353.6399999995|
|2009-09-01 00:00:00|        648992.75|
|2009-10-01 00:00:00|570492.5099999999|
|2009-11-01 00:00:00|566171.2399999998|
|2009-12-01 00:00:00|560232.9200000002|
|2010-01-01 00:00:00|577908.3700000001|
|2010-02-01 00:00:00|586651.7600000002|
|2010-03-01 00:00:00|529839.6000000002|
|2010-04-01 00:00:00|490967.0900000001|
|2010-05-01 00:00:00|748098.0000000003|
|2010-06-01 00:00:00|611528.2600000001|
|2010-07-01 00:00:00|584327.2599999997|
|2010-08-01 00:00:00|622679.6900000004|
+-------------------+-----------------+
only showing top 20 rows



In [10]:
summary = summary.withColumn("OrderMonthYear", summary['OrderMonthYear'].substr(0,10))

In [11]:
results = summary\
.rdd\
.map(lambda r: (int(r['OrderMonthYear'].replace('-','')), r['sum(SaleAmount)']))\
.toDF()\
.selectExpr("_1 as OrderMonthYear","_2 as sum")

In [12]:
results.show()

+--------------+-----------------+
|OrderMonthYear|              sum|
+--------------+-----------------+
|      20090101|734559.3599999996|
|      20090201|539887.7999999998|
|      20090301|559449.9600000002|
|      20090401|        614983.31|
|      20090501|637481.3899999998|
|      20090601|555516.2199999999|
|      20090701|        670807.13|
|      20090801|660353.6399999995|
|      20090901|        648992.75|
|      20091001|570492.5099999999|
|      20091101|566171.2399999998|
|      20091201|560232.9200000002|
|      20100101|577908.3700000001|
|      20100201|586651.7600000002|
|      20100301|529839.6000000002|
|      20100401|490967.0900000001|
|      20100501|748098.0000000003|
|      20100601|611528.2600000001|
|      20100701|584327.2599999997|
|      20100801|622679.6900000004|
+--------------+-----------------+
only showing top 20 rows



In [13]:
from pyspark.mllib.regression import LabeledPoint

In [28]:
data = results\
.select("OrderMonthYear", "sum")\
.rdd\
.map(lambda r: LabeledPoint(r[1], [r[0]]))

In [23]:
from pyspark.mllib.regression import LinearRegressionWithSGD

In [25]:
lr = LinearRegressionWithSGD()

In [33]:
modelA = lr.train(data,intercept=True)

In [20]:
spark.stop()

In [35]:
data.toDF().show()

+-------------+-----------------+
|     features|            label|
+-------------+-----------------+
|[2.0090101E7]|734559.3599999996|
|[2.0090201E7]|539887.7999999998|
|[2.0090301E7]|559449.9600000002|
|[2.0090401E7]|        614983.31|
|[2.0090501E7]|637481.3899999998|
|[2.0090601E7]|555516.2199999999|
|[2.0090701E7]|        670807.13|
|[2.0090801E7]|660353.6399999995|
|[2.0090901E7]|        648992.75|
|[2.0091001E7]|570492.5099999999|
|[2.0091101E7]|566171.2399999998|
|[2.0091201E7]|560232.9200000002|
|[2.0100101E7]|577908.3700000001|
|[2.0100201E7]|586651.7600000002|
|[2.0100301E7]|529839.6000000002|
|[2.0100401E7]|490967.0900000001|
|[2.0100501E7]|748098.0000000003|
|[2.0100601E7]|611528.2600000001|
|[2.0100701E7]|584327.2599999997|
|[2.0100801E7]|622679.6900000004|
+-------------+-----------------+
only showing top 20 rows



In [34]:
modelA

(weights=[nan], intercept=nan)

In [42]:
valuesAndPreds = data.map(lambda p: (p['label'], float(modelA.predict(p['features']))))

In [43]:
valuesAndPreds.toDF().show()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 887.0 failed 1 times, most recent failure: Lost task 0.0 in stage 887.0 (TID 11614, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/chengyinliu/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/Users/chengyinliu/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/chengyinliu/spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pyspark/rdd.py", line 1354, in takeUpT

In [48]:
modelA.predict(data_temp['features'])

TypeError: Cannot convert type <class 'pyspark.sql.column.Column'> into Vector

In [47]:
data_temp = data.toDF()

In [49]:
data_temp['features']

Column<features>

In [50]:
from pyspark.ml.feature import VectorAssembler

In [55]:
vecAssembler = VectorAssembler(inputCols=['features'], outputCol="features")
stream_df = vecAssembler.transform(data_temp)

IllegalArgumentException: u'Data type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> of column features is not supported.'